In [4]:
import os 

os.environ["GOOGLE_API_KEY"]= "
os.environ["GROQ_API_KEY"] = "

In [40]:
!pip install sentence-transformers

  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 MB 2.5 MB/s eta 0:00:0000:0100:08
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:0000:0100:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.4 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.1 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [7]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro")
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros:**

* **Easy to learn and use:** Python has a simple and intuitive syntax, making it accessible to beginners and experienced programmers alike.
* **Versatile:** Python can be used for a wide range of applications, including web development, data science, machine learning, and scripting.
* **Extensive library support:** Python has a vast ecosystem of open-source libraries, providing additional functionality for various tasks.
* **Interpreted:** Python is an interpreted language, which allows for rapid development and testing.
* **Cross-platform:** Python can run on multiple platforms, including Windows, macOS, and Linux.
* **Object-oriented:** Python supports object-oriented programming principles, offering code reusability and maintainability.
* **Active community:** Python has a large and active community, providing support and resources for developers.

**Cons:**

* **Slow execution speed:** Python is generally slower than compiled languages like C++ or Java.
* **Memory-intens

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", )

In [32]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import os
import random

from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

#google_api_key = os.getenv('GOOGLE_API_KEY')
#groq_api_key = os.getenv('GROQ_API_KEY')


llm = ChatGroq(groq_api_key= "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", model_name="Llama3-8b-8192")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

loader = PyPDFDirectoryLoader("/home/ishan/Projects/QA_System/Data/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)
final_documents = text_splitter.split_documents(docs)
vectors = FAISS.from_documents(final_documents, embeddings)

prompt = ChatPromptTemplate.from_template(
    """
    Please provide the most accurate response based on the question and make it super easy to understand like you are explaining to a 5 years old and make it sound like a story.
    <context>
    {context}
    <context>
    Questions:{input}
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

random_page = random.choice(final_documents)
random_fact_prompt = f"Tell a random fact about exoplanet from page {random_page}."

response = retrieval_chain.invoke({'input': random_fact_prompt})
print(response['answer'])

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import os
import random

from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import Qdrant

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize the LLM and embeddings
llm = ChatGroq(groq_api_key="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", model_name="Llama3-8b-8192")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Load PDF documents
loader = PyPDFDirectoryLoader("/home/ishan/Projects/QA_System/Data/")
docs = loader.load()

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)
final_documents = text_splitter.split_documents(docs)

# Initialize Qdrant client and create a collection
qdrant_client = QdrantClient(host="localhost", port=6333)
collection_name = "exoplanet_qa_system"


vectors = Qdrant.from_documents(
    documents=final_documents, 
    embedding=embeddings, 
    client=qdrant_client
)

# Create the prompt template
prompt = ChatPromptTemplate.from_template(
    """
    Please provide the most accurate response based on the question and make it super easy to understand like you are explaining to a 5-year-old and make it sound like a story.
    <context>
    {context}
    <context>
    Questions: {input}
    """
)

# Create document and retrieval chains
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Get a random page from the documents and form the query
random_page = random.choice(final_documents)
random_fact_prompt = f"Tell a random fact about exoplanet from page {random_page}."

# Invoke the retrieval chain and print the response
response = retrieval_chain.invoke({'input': random_fact_prompt})
print(response['answer'])
